# [Module 1.2] 세이지 메이커 로컬 모드 및 스크립트 모드로 훈련

본 워크샵의 모든 노트북은 **<font color="red">conda_tensorflow2_p36</font>** 를 사용합니다.

이 노트북은 아래와 같은 작업을 합니다.
- 1. 기본 환경 세팅 
- 2. 노트북에서 세이지 메이커 스크립트 모드 스타일로 코드 변경
- 3. 세이지 메이커 스크립트 모드 스타일로 코드 실행 (실제로 세이지 메이커 사용 안함)
- 4. 세이지 메이커 로컬 모드로 훈련
- 5. 세이지 메이커의 호스트 모드로 훈련
- 6. 모델 아티펙트 경로 저장

## 참고:
- 이 페이지를 보시면 Cifar10 데이터 설명 및 기본 모델 훈련이 있습니다. --> [Train a Keras Sequential Model (TensorFlow 2.0)](https://github.com/daekeun-ml/tensorflow-in-sagemaker-workshop/blob/master/0_Running_TensorFlow_In_SageMaker_tf2.ipynb)
    - 메인 깃 리파지토리: [SageMaker Workshop: Tensorflow-Keras 모델을 Amazon SageMaker에서 학습하기](https://github.com/daekeun-ml/tensorflow-in-sagemaker-workshop)

---

# 1. 기본 환경 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-cnn-cifar10"

role = sagemaker.get_execution_role()


In [3]:
import tensorflow as tf
print("tensorflow version: ", tf.__version__)

tensorflow version:  2.4.1


In [4]:
%store -r train_dir
%store -r validation_dir
%store -r eval_dir
%store -r data_dir

# 2. 노트북에서 세이지 메이커 스크립트 모드 스타일로 코드 변경
- 이 페이지를 보시면 기본 코드를 세이지 메이커 스크립트 모드로 변경하는 내용이 있습니다.
    - [Train a Keras Sequential Model (TensorFlow 2.0)](https://github.com/daekeun-ml/tensorflow-in-sagemaker-workshop/blob/master/0_Running_TensorFlow_In_SageMaker_tf2.ipynb)


- 아래의 `# !pygmentize src/cifar10_keras_sm_tf2.py` 의 주석을 제거하시고 보시면 실제 변경된 코드가 있습니다.


In [5]:
# !pygmentize src/cifar10_keras_sm_tf2.py

# 3. 세이지 메이커 스크립트 모드 스타일로 코드 실행 (실제로 세이지 메이커 사용 안함)

테스트를 위해 위와 동일한 명령(command)으로 새 스크립트를 실행하고, 예상대로 실행되는지 확인합니다. <br>
SageMaker TensorFlow API 호출 시에 환경 변수들은 자동으로 넘겨기지만, 로컬 주피터 노트북에서 테스트 시에는 수동으로 환경 변수들을 지정해야 합니다. (아래 예제 코드를 참조해 주세요.)

```python
%env SM_MODEL_DIR=./logs
```

In [6]:
print("train_dir: ", train_dir)
print("validation_dir: ", validation_dir)
print("eval_dir: ", eval_dir)

train_dir:  data/cifar10/train
validation_dir:  data/cifar10/validation
eval_dir:  data/cifar10/eval


In [7]:
%%time
!mkdir -p logs   

# Number of GPUs on this machine
%env SM_NUM_GPUS=1
# Where to save the model
%env SM_MODEL_DIR=./logs

!python src/cifar10_keras_sm_tf2.py --model_dir ./logs \
                                                --train {train_dir} \
                                                --validation {validation_dir} \
                                                --eval {eval_dir} \
                                                --epochs 1
!rm -rf logs

env: SM_NUM_GPUS=1
env: SM_MODEL_DIR=./logs
args: 
 Namespace(batch_size=128, epochs=1, eval='data/cifar10/eval', learning_rate=0.001, model_dir='./logs', model_output_dir='./logs', momentum=0.9, optimizer='adam', train='data/cifar10/train', validation='data/cifar10/validation', weight_decay=0.0002)
312/312 [==============================] - 76s 237ms/step - loss: 2.2003 - accuracy: 0.2331 - val_loss: 1.8217 - val_accuracy: 0.3403
INFO:tensorflow:Assets written to: ./logs/1/assets
INFO:tensorflow:Assets written to: ./logs/1/assets
CPU times: user 2.05 s, sys: 316 ms, total: 2.37 s
Wall time: 1min 27s


# 4. 세이지 메이커 로컬 모드로 훈련

본격적으로 학습을 시작하기 전에 로컬 모드를 사용하여 디버깅을 먼저 수행합니다. 로컬 모드는 학습 인스턴스를 생성하는 과정이 없이 로컬 인스턴스로 컨테이너를 가져온 후 곧바로 학습을 수행하기 때문에 코드를 보다 신속히 검증할 수 있습니다.

Amazon SageMaker Python SDK의 로컬 모드는 TensorFlow 또는 MXNet estimator서 단일 인자값을 변경하여 CPU (단일 및 다중 인스턴스) 및 GPU (단일 인스턴스) SageMaker 학습 작업을 에뮬레이션(enumlate)할 수 있습니다. 

로컬 모드 학습을 위해서는 docker-compose 또는 nvidia-docker-compose (GPU 인스턴스인 경우)의 설치가 필요합니다. 아래 코드 셀을 통해 본 노트북 환경에 docker-compose 또는 nvidia-docker-compose를 설치하고 구성합니다. 
 
로컬 모드의 학습을 통해 여러분의 코드가 현재 사용 중인 하드웨어를 적절히 활용하고 있는지 확인하기 위한 GPU 점유와 같은 지표(metric)를 쉽게 모니터링할 수 있습니다.

In [8]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json    
!/bin/bash ./local_mode_setup.sh

nvidia-docker2 already installed. We are good to go!
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


### 로컬 모드로 훈련 실행
- 아래의 두 라인이 로컬모드로 훈련을 지시 합니다.
```python
    instance_type=instance_type, # local_gpu or local 지정
    session = sagemaker.LocalSession(), # 로컬 세션을 사용합니다.
```

#### 로컬의 GPU, CPU 여부로 instance_type 결정

In [9]:
import os
import subprocess


instance_type = "local_gpu" # GPU 사용을 가정 합니다. CPU 사용시에 'local' 로 정의 합니다.

print("Instance type = " + instance_type)

Instance type = local_gpu


학습 작업을 시작하기 위해 `estimator.fit() ` 호출 시, Amazon ECR에서 Amazon SageMaker TensorFlow 컨테이너를 로컬 노트북 인스턴스로 다운로드합니다.

`sagemaker.tensorflow` 클래스를 사용하여 SageMaker Python SDK의 Tensorflow Estimator 인스턴스를 생성합니다.
인자값으로 하이퍼파라메터와 다양한 설정들을 변경할 수 있습니다.


자세한 내용은 [documentation](https://sagemaker.readthedocs.io/en/stable/using_tf.html#training-with-tensorflow-estimator)을 확인하시기 바랍니다.

In [10]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='src',
                       role=role,
                       framework_version='2.4.1',
                       py_version='py37',
                       script_mode=True,
                       hyperparameters={'epochs' : 1},
                       train_instance_count=1, 
                       train_instance_type= instance_type)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
%%time
estimator.fit({'train': f'file://{train_dir}',
               'validation': f'file://{validation_dir}',
               'eval': f'file://{eval_dir}'})

Creating roqiryt46i-algo-1-jt4ft ... 
Creating roqiryt46i-algo-1-jt4ft ... done
Attaching to roqiryt46i-algo-1-jt4ft
roqiryt46i-algo-1-jt4ft | 2021-10-11 09:57:34.729714: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
roqiryt46i-algo-1-jt4ft | 2021-10-11 09:57:34.729908: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
roqiryt46i-algo-1-jt4ft | 2021-10-11 09:57:34.734209: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
roqiryt46i-algo-1-jt4ft | 2021-10-11 09:57:34.771766: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
roqiryt46i-algo-1-jt4ft | 2021-10-11 09:57:36,568 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
roqiryt46i-algo

## ECR 로 부터 로컬에 다운로드된 도커 이미지 확인

In [12]:
! docker image ls

REPOSITORY                                                         TAG                 IMAGE ID            CREATED             SIZE
763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training   2.4.1-gpu-py37      8467bc1c5070        5 months ago        8.91GB


# 5. 세이지 메이커의 호스트 모드로 훈련

### 데이터 세트를 S3에 업로드

- 로컬에 저장되어 있는 데이터를 S3 로 업로드하여 사용합니다.

In [15]:
dataset_location = sagemaker_session.upload_data(path=data_dir, key_prefix='data/DEMO-cifar10')
display(dataset_location)

's3://sagemaker-us-east-1-227612457811/data/DEMO-cifar10'

In [16]:
from sagemaker.tensorflow import TensorFlow
estimator = TensorFlow(base_job_name='cifar10',
                       entry_point='cifar10_keras_sm_tf2.py',
                       source_dir='src',
                       role=role,
                       framework_version='2.4.1',
                       py_version='py37',
                       script_mode=True,                       
                       hyperparameters={'epochs': 5},
                       train_instance_count=1, 
                       train_instance_type='ml.p2.8xlarge')

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


## SageMaker Host Mode 로 훈련
- `cifar10_estimator.fit(inputs, wait=False)`
    - 입력 데이터를 inputs로서 S3 의 경로를 제공합니다.
    - wait=False 로 지정해서 async 모드로 훈련을 실행합니다. 
        - 실행 경과는 아래의 cifar10_estimator.logs() 에서 확인 합니다.

In [19]:
%%time
estimator.fit({'train':'{}/train'.format(dataset_location),
              'validation':'{}/validation'.format(dataset_location),
              'eval':'{}/eval'.format(dataset_location)}, wait=False)

CPU times: user 64.4 ms, sys: 0 ns, total: 64.4 ms
Wall time: 320 ms


In [20]:
estimator.logs()

2021-10-11 10:10:06 Starting - Starting the training job...
2021-10-11 10:10:29 Starting - Launching requested ML instancesProfilerReport-1633947005: InProgress
............
2021-10-11 10:12:29 Starting - Preparing the instances for training.........
2021-10-11 10:14:02 Downloading - Downloading input data
2021-10-11 10:14:02 Training - Downloading the training image..................
2021-10-11 10:17:00 Training - Training image download completed. Training in progress.2021-10-11 10:17:01.282337: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-10-11 10:17:01.288520: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-10-11 10:17:01.384938: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-11 10:17:01.489458: W tenso

# 6. 모델 아티펙트 저장
- S3 에 저장된 모델 아티펙트를 저장하여 추론시 사용합니다.

In [22]:
keras_script_artifact_path = estimator.model_data
print("script_artifact_path: ", keras_script_artifact_path)

%store keras_script_artifact_path

script_artifact_path:  s3://sagemaker-us-east-1-227612457811/cifar10-2021-10-11-10-10-05-339/output/model.tar.gz
Stored 'keras_script_artifact_path' (str)
